In [1]:
pwd()

'/home/ifte/amiebot_project/AI-system/Models/Seq2SeqWordModel'

In [2]:
# to install mecab
# sudo apt install mecab mecab-ipadic-utf8
import MeCab
wakati = MeCab.Tagger("-Owakati")



### Datasets
http://www.manythings.org/anki/  (Download and unzip jap-eng.zip file)

In [3]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.
/home/ifte/amiebot_project/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ifte/amiebot_project/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ifte/amiebot_project/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ifte/amiebot_project/lib/python3.6/sit

In [4]:
lines= pd.read_table('jpn.txt', names=['eng', 'jap'])

In [5]:
lines.shape

(45093, 2)

In [6]:
lines.tail(5)
lines.jap.tail(5)

45088    「恋人にさよならのキスくらいしないと、キスしてあばよ、ということになるぞ」とトムは言ったが、...
45089    日本ではリヒター・スケールでマグニチュード5.0以上の余震が今週5回ありました。しかし科学者...
45090    ただ今到着のバスは、国内線第1ターミナル行きです。国際線ターミナルにお越しの方は、しばらくそ...
45091    ネイティブの子どもは、何年も学んだ非ネイティブが知らず今後も知り得ないたくさんのことを自身の...
45092    色々並行してやってるから芥川ばかり読んでるのでもないのだよ。今は英語読んでる時間が増えてる。...
Name: jap, dtype: object

In [7]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
#lines.jap=lines.jap.apply(lambda x: x.lower())
lines.jap = lines.jap.apply(lambda x: wakati.parse(x).strip("\n"))


In [8]:
lines.tail(5)
lines.jap.tail(5)

45088    「 恋人 に さよなら の キス くらい し ない と 、 キス し て あ ば よ 、 と...
45089    日本 で は リヒター ・ スケール で マグニチュード 5 . 0 以上 の 余震 が 今...
45090    ただ 今 到着 の バス は 、 国内線 第 1 ターミナル 行き です 。 国際線 ターミ...
45091    ネイティブ の 子ども は 、 何 年 も 学ん だ 非 ネイティブ が 知ら ず 今後 も...
45092    色々 並行 し て やっ てる から 芥川 ばかり 読ん でる の で も ない の だ よ...
Name: jap, dtype: object

In [9]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.jap=lines.jap.apply(lambda x: re.sub("'", '', x))

In [10]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.jap=lines.jap.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [11]:
# Remove all numbers from text
#remove_digits = str.maketrans('', '', digits)
#lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
#lines.jap = lines.jap.apply(lambda x: re.sub("[123456789]", "", x))

In [12]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.jap=lines.jap.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.jap=lines.jap.apply(lambda x: re.sub(" +", " ", x))

In [13]:
# Add start and end tokens to target sequences
lines.jap = lines.jap.apply(lambda x : 'START '+ x + ' END')

In [14]:
lines.sample(10)

,eng,jap
37801,she tried to prevent the rumor from spreading,START 彼女 は その うわさ が 広がる の を 防ご う と 努力 し た 。 END
11919,the food was very good,START 食事 は 非常 に よかっ た 。 END
26496,he thinks that he is a great poet,START 彼 は 自分 の こと を 偉大 な 詩人 だ と 思っ て いる 。 END
23181,this material stretches easily,START この 生地 は 引っ張る と すぐ 伸びる 。 END
35187,the train was late because of an accident,START 列車 は 事故 の ため 遅れ た 。 END
664,whos she,START 彼女 は 誰 です か 。 END
24881,all the students passed the test,START 生徒 は 全員 テスト に 合格 し まし た 。 END
8372,he broke his promise,START 彼 は 約束 を 破っ た 。 END
4183,how much is that,START いくら です か ？ END
9648,go back to your seats,START 自分 の 席 に もどり なさい 。 END


In [15]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Japanese 
all_japanese_words=set()
for jap in lines.jap:
    for word in jap.split():
        if word not in all_japanese_words:
            all_japanese_words.add(word)

In [16]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

39

In [17]:
# Max Length of target sequence
lenght_list=[]
for l in lines.jap:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

60

In [18]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_japanese_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_japanese_words)
num_encoder_tokens, num_decoder_tokens

(9386, 13905)

In [19]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

13906

In [20]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [21]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [22]:
lines = shuffle(lines)
lines.head(10)

,eng,jap
16152,that big one is also mine,START あの 大きい の も 私 の です 。 END
5105,hes studying now,START 彼 は 勉強 中 です 。 END
28398,thank you very much for everything,START いろいろ お世話 に なり まし た 。 END
38128,he will have to undergo an operation next week,START 来週 彼 は 手術 を 受け なけれ ば なら ない 。 END
24994,give me a glass of water please,START 私 に 水 を １ 杯 下さい 。 END
44450,it is absurd to believe that young children ca...,START 幼い 子供 が 警官 より 走る の が 速い と 信じる の は 理屈 に 合...
5691,tom chews tobacco,START トム は 噛み タバコ を 使用 する 。 END
36513,she had never dreamed of meeting him abroad,START まさか 海外 で 彼 に 会う と は 、 彼女 は 夢にも 思わ なかっ た ...
9948,i cant wait any more,START もう 待て ない よ 。 END
19741,how can you be so optimistic,START よくも そう 楽天的 で い られる よ 。 END


In [23]:
# Train - Test Split
X, y = lines.eng, lines.jap
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((40583,), (4510,))

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [24]:
X_train.to_pickle('Weights/X_train.pkl')
X_test.to_pickle('Weights/X_test.pkl')

In [25]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [26]:
latent_dim = 50

In [27]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
#from IPython.display import Image
#Image(retina=True, filename='train_model.png')

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

### Always remember to save the weights

In [ ]:
model.save_weights('nmt_weights.h5')

### Load the weights, if you close the application

In [ ]:
model.load_weights('nmt_weights.h5')

### Inference Setup

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
Image(retina=True, filename='encoder_model.png')

In [ ]:
Image(retina=True, filename='decoder_model.png')

### Decode sample sequeces

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

### Evaluation on Validation Dataset

In [ ]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Japanese Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])